In [2]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 불러오기 (예시: pandas DataFrame)
data = pd.read_csv('apt_all_numeric_information.csv')

# 데이터 준비
X = data.drop(columns=['adjusted_price', 'NO'])  # 입력 변수
y = data['adjusted_price']                       # 목표 변수

# 데이터셋 나누기: 학습용(train), 검증용(valid), 테스트용(test)
train_X, temp_X, train_y, temp_y = train_test_split(X, y, test_size=0.3, random_state=42)  # train: 70%, temp: 30%
valid_X, test_X, valid_y, test_y = train_test_split(temp_X, temp_y, test_size=0.5, random_state=42)  # valid: 15%, test: 15%

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(train_X, label=train_y)
valid_data = lgb.Dataset(valid_X, label=valid_y)

# LightGBM 하이퍼파라미터 설정
params = {
    'objective': 'regression',  # 회귀 문제
    'metric': 'rmse',          # 평가 지표
    'boosting_type': 'gbdt',   # 부스팅 방식
    'num_leaves': 31,          # 하나의 트리에서 가질 수 있는 최대 잎 노드 수
    'learning_rate': 0.05,     # 학습률
    'feature_fraction': 0.9    # 학습에 사용할 피처 비율
}

In [5]:

# LightGBM 모델 학습
model = lgb.test(
    params,
    train_data,
    valid_sets=[train_data, valid_data],  # 학습 데이터와 검증 데이터
    num_boost_round=1000,
    early_stopping_rounds=50,            # 조기 종료 조건
    verbose_eval=100                     # 로그 출력 주기
)

# 테스트 데이터에서 예측 수행
y_test_pred = model.predict(test_X, num_iteration=model.best_iteration)

# 평가 지표 계산
rmse_test = root_mean_squared_error(test_y, y_test_pred, squared=False)  # RMSE
mae_test = mean_absolute_error(test_y, y_test_pred)                # MAE

print("Test RMSE:", rmse_test)
print("Test MAE:", mae_test)


AttributeError: module 'lightgbm' has no attribute 'test'